In [ ]:
import numpy as np
from scipy.integrate import quad

# Constants
pi = np.pi
c = 299792458
h = 6.62607015e-34
hbar = h / (2 * pi)
e_charge = 1.60217663e-19
m_e = 9.10938356e-31
epsilon_0 = 8.85418781e-12
alpha = e_charge**2 / (4 * pi * epsilon_0 * hbar * c)

# 1. Standard Fine Structure Calculation (Dirac/Sommerfeld)
# Energy levels: E_nj = E_n [ 1 + (alpha^2 / n^2) * (n / (j + 0.5) - 3/4) ]
# n=2
# Level 1 (j=1/2): 2S_1/2, 2P_1/2
# Level 2 (j=3/2): 2P_3/2
# Splitting Delta E_FS = E(j=3/2) - E(j=1/2) (Magnitude)

E_Rydberg_eV = 13.6056931396 # Ionization energy of Hydrogen
E_n_eV = E_Rydberg_eV / (2**2) # Base energy for n=2 (~3.4 eV)

# Correction terms (dimensionless, multiplying E_n)
# Term = (alpha^2 / n^2) * (n / (j + 0.5) - 3/4)
term_j1_2 = (alpha**2 / 4) * (2 / 1 - 3/4) # j=1/2 -> j+0.5 = 1
term_j3_2 = (alpha**2 / 4) * (2 / 2 - 3/4) # j=3/2 -> j+0.5 = 2

diff_FS_dimensionless = abs(term_j1_2 - term_j3_2)
diff_FS_eV = E_n_eV * diff_FS_dimensionless

# 2. R.Q.M. Calculation (Geometric Precession)
# We calculate the phase shift per orbit, then convert to energy.
# Shift_E = E_n * (Delta_Phi_Average / 2pi)

def calculate_rqm_shift(n, k):
    # Parameters
    e_q = np.sqrt(1 - (k/n)**2)
    # Scale factors: In RQM, kappa^2 ~ 1/r. Precession ~ kappa^4 / beta^2.
    # At perihelion (or any point), we need the explicit formula.
    # Delta_phi(o) = (3pi/2) * (kappa(o)^4 / beta(o)^2)
    # kappa(o)^2 = R_q / r(o)
    # beta(o)^2 = kappa(o)^2 - 2|W|
    # W = alpha^2 / (2n^2) (dimensionless in natural units where c=1 etc, but let's stick to ratios)

    # Let's work with dimensionless ratios to be safe.
    # r_n = n^2 (in units of a_0 approx, but let's use the ratio R_q / a_0)
    # R_q / a_0 = 2 * alpha^2.
    # So r(in units of a_0) = (n^2) * ...
    # Let's use x = r / a_0.
    # R_q_dim = 2 * alpha**2
    # W_dim = alpha**2 / (2 * n**2)

    # r(phase o) in units of a_0:
    # r_n_dim = n**2
    # r_dim = r_n_dim * (1 - e_q**2) / (1 + e_q * np.cos(phase))

    def get_integrands(phase):
        r_dim = (n**2) * (1 - e_q**2) / (1 + e_q * np.cos(phase))

        # Local Projections
        kappa_sq = (2 * alpha**2) / r_dim
        beta_sq = kappa_sq - (alpha**2 / n**2)

        # Precession Rate (dPhi/dPhase is not correct, this is dPhi/Orbit? No, dPhi instantaneous)
        # The formula 3pi/2 * ... is the accumulated precession per orbit if parameters were constant?
        # No, in General Relativity WILL Part I, Delta_phi = 3pi/2 * ... is the shift PER ORBIT for given parameters.
        # But here parameters change.
        # We need the instantaneous precession rate d(Phi)/dt or d(Phi)/d(phase).
        # Let's assume the formula given dPhi = (3/2) * (Rs/r) * d(theta) is the instantaneous GR shift?
        # In WILL Part I: Delta_phi = 6pi * (GM/c^2 * 1/L).
        # Let's stick to the user's defined "Lamb Generator": Delta_phi(x) = 3pi/2 * K^4 / B^2.
        # This looks like an amplitude. Let's assume this is "Precession per radian of orbit"?
        # Or "Precession per full orbit if frozen"?
        # User script in Desmos: Shift = Integral(Delta * w) / Integral(w).
        # This implies Delta_phi(x) is a local value, averaged over the orbit.
        # Let's use exactly that.

        local_prec = (3 * pi / 2) * (kappa_sq**2) / beta_sq

        # Weight (1/r^2 for time? No, 1/omega ~ r^2)
        # In Desmos user used w(x) = 1 / (1 + e cos)^2.
        # r ~ 1/(1+e cos). So r^2 ~ 1/(1+e cos)^2.
        # So Weight is proportional to r^2.
        weight = 1 / (1 + e_q * np.cos(phase))**2

        return local_prec * weight, weight

    # Integrate 0 to 2pi
    val_num, _ = quad(lambda x: get_integrands(x)[0], 0, 2*pi)
    val_den, _ = quad(lambda x: get_integrands(x)[1], 0, 2*pi)

    avg_precession = val_num / val_den
    return avg_precession

# Calculate for k=1 (Elliptical, ~2S/2P_1/2) and k=2 (Circular, ~2P_3/2)
shift_k1_rads = calculate_rqm_shift(2, 1)
shift_k2_rads = calculate_rqm_shift(2, 2)

# Convert to Energy
# Energy Shift = E_n * (Shift_Rads / 2pi)
E_shift_k1 = E_n_eV * (shift_k1_rads / (2 * pi))
E_shift_k2 = E_n_eV * (shift_k2_rads / (2 * pi))

diff_RQM_eV = abs(E_shift_k1 - E_shift_k2)

print(f"{diff_FS_eV=}")
print(f"{diff_RQM_eV=}")
print(f"Ratio RQM/FS: {diff_RQM_eV / diff_FS_eV}")

diff_FS_eV=4.528259888092377e-05
diff_RQM_eV=0.00013584779664277212
Ratio RQM/FS: 3.0000000000000178


In [ ]:
import numpy as np
from scipy.integrate import quad

pi = np.pi

# Physical constants (only used for alpha; energy scale uses Rydberg eV below)
c = 299792458
h = 6.62607015e-34
hbar = h / (2 * pi)
e_charge = 1.60217663e-19
epsilon_0 = 8.85418781e-12
alpha = e_charge**2 / (4 * pi * epsilon_0 * hbar * c)

# Rydberg scale in eV (your script already uses this)
E_Rydberg_eV = 13.6056931396

def dirac_fs_splitting_eV(n, j_low=1/2, j_high=3/2):
    """
    Same approximate Dirac/Sommerfeld correction form you used,
    generalized to arbitrary n. (Still an approximation.)
    """
    E_n_eV = E_Rydberg_eV / (n**2)
    term_low  = (alpha**2 / (n**2)) * (n / (j_low  + 0.5) - 3/4)
    term_high = (alpha**2 / (n**2)) * (n / (j_high + 0.5) - 3/4)
    return abs(term_low - term_high) * E_n_eV

def rqm_avg_precession(n, k, weight_mode="time", kappa_coeff=np.sqrt(2), eps=1e-18):
    """
    Computes <DeltaPhi>_{n,k} = ∮ DeltaPhi(o) w(o) do / ∮ w(o) do
    with controllable:
      - weight_mode: "time" (proportional to r^2 -> your current), or "phase" (uniform)
      - kappa_coeff: sets kappa_q = (kappa_coeff * alpha / n), i.e. replaces sqrt(2) if desired
    """
    if not (1 <= k <= n):
        raise ValueError("Require 1 <= k <= n")

    # Your quantum eccentricity analog
    e_q = np.sqrt(max(0.0, 1.0 - (k/n)**2))

    # In your dimensionless setup:
    # R_q / a0 = 2 * alpha^2  (your script uses this)
    # Here we allow generalization consistent with kappa_q = kappa_coeff * alpha/n:
    # For circular reference at r_dim = n^2: kappa_sq = (kappa_coeff^2 * alpha^2) / n^2.
    # Your original code hard-coded kappa_coeff^2 = 2 via (2*alpha^2)/r_dim.
    # So replace 2 by (kappa_coeff^2).
    kappa_coeff_sq = float(kappa_coeff**2)

    # Your energy invariant (dimensionless): W_q = alpha^2/(2 n^2)
    # Keep as-is (your definition). This is part of the test; we’re not “fixing” anything here.
    W2 = (alpha**2 / (n**2))  # this equals 2|W_q| in your beta_sq formula

    def local_terms(phase):
        # r(o) in units of a0 (same as your code)
        r_dim = (n**2) * (1 - e_q**2) / (1 + e_q * np.cos(phase))

        # Local kappa^2
        kappa_sq = (kappa_coeff_sq * alpha**2) / r_dim

        # Local beta^2 per your definition: beta^2 = kappa^2 - alpha^2/n^2
        # (same as your code: beta_sq = kappa_sq - W2)
        beta_sq = kappa_sq - W2

        # Guard: if beta_sq <= 0 somewhere, the integrand blows up (unphysical / inconsistent region)
        if beta_sq <= eps:
            return np.inf, np.inf, np.inf

        # Your local precession generator
        local_prec = (3 * pi / 2) * (kappa_sq**2) / beta_sq

        # Weights
        if weight_mode == "time":
            # proportional to r^2 -> since r ~ 1/(1+e cos), r^2 ~ 1/(1+e cos)^2
            w = 1.0 / (1 + e_q * np.cos(phase))**2
        elif weight_mode == "phase":
            w = 1.0
        else:
            raise ValueError("weight_mode must be 'time' or 'phase'")

        return local_prec, w, beta_sq

    def num_integrand(x):
        prec, w, _ = local_terms(x)
        return prec * w

    def den_integrand(x):
        _, w, _ = local_terms(x)
        return w

    val_num, _ = quad(num_integrand, 0, 2*pi, limit=400)
    val_den, _ = quad(den_integrand, 0, 2*pi, limit=400)

    return val_num / val_den

def rqm_energy_shift_eV(n, k, weight_mode="time", kappa_coeff=np.sqrt(2)):
    """
    DeltaE_geom(n,k) = |E_n| * <DeltaPhi> / (2pi)
    using |E_n| = Rydberg/n^2 (same as your current choice).
    """
    E_n_eV = E_Rydberg_eV / (n**2)
    avg_phi = rqm_avg_precession(n, k, weight_mode=weight_mode, kappa_coeff=kappa_coeff)
    return E_n_eV * (avg_phi / (2*pi))

def run_tests(n_list=(2,3,4,5), pairs="edge", weight_mode="time", kappa_coeff=np.sqrt(2)):
    """
    pairs:
      - "edge": compare k=1 vs k=n  (max-elliptic vs circular)
      - "adjacent": compare k=n-1 vs k=n  (near-circular split)
      - list of tuples: [(k1,k2),...]
    """
    print(f"\n=== TESTS: weight_mode={weight_mode}, kappa_coeff={kappa_coeff} ===")
    for n in n_list:
        if pairs == "edge":
            k1, k2 = 1, n
        elif pairs == "adjacent":
            if n < 2:
                continue
            k1, k2 = n-1, n
        else:
            # user-provided list per n not supported here; keep simple
            raise ValueError("pairs must be 'edge' or 'adjacent' in this minimal runner")

        E1 = rqm_energy_shift_eV(n, k1, weight_mode=weight_mode, kappa_coeff=kappa_coeff)
        E2 = rqm_energy_shift_eV(n, k2, weight_mode=weight_mode, kappa_coeff=kappa_coeff)
        diff_RQM = abs(E1 - E2)

        # Keep your FS comparator as a reference number (even if you don’t care about its ontology)
        diff_FS = dirac_fs_splitting_eV(n)

        print(f"n={n:2d}  pair(k1,k2)=({k1},{k2})  diff_RQM_eV={diff_RQM:.16e}  diff_FS_eV={diff_FS:.16e}  ratio(RQM/FS)={diff_RQM/diff_FS:.16e}")

# --------- Run the exact reproduction case (n=2, k=1 vs 2) ----------
def reproduce_original():
    n=2
    E1 = rqm_energy_shift_eV(n, 1, weight_mode="time", kappa_coeff=np.sqrt(2))
    E2 = rqm_energy_shift_eV(n, 2, weight_mode="time", kappa_coeff=np.sqrt(2))
    diff_RQM = abs(E1 - E2)
    diff_FS  = dirac_fs_splitting_eV(n)
    print("\n=== REPRODUCE ORIGINAL (n=2, k=1 vs 2) ===")
    print(f"diff_FS_eV={diff_FS}")
    print(f"diff_RQM_eV={diff_RQM}")
    print(f"Ratio RQM/FS: {diff_RQM/diff_FS}")

reproduce_original()

# --------- TEST A: different n, same weighting, same sqrt(2) ----------
run_tests(n_list=(2,3,4,5), pairs="edge", weight_mode="time", kappa_coeff=np.sqrt(2))
run_tests(n_list=(2,3,4,5), pairs="adjacent", weight_mode="time", kappa_coeff=np.sqrt(2))

# --------- TEST B: change weighting (kills the r^2 cancellation) ----------
run_tests(n_list=(2,3,4,5), pairs="edge", weight_mode="phase", kappa_coeff=np.sqrt(2))

# --------- TEST C: change kappa scaling constant (breaks the built-in sqrt(2)) ----------
run_tests(n_list=(2,3,4,5), pairs="edge", weight_mode="time", kappa_coeff=1.40)
run_tests(n_list=(2,3,4,5), pairs="edge", weight_mode="time", kappa_coeff=1.45)


=== REPRODUCE ORIGINAL (n=2, k=1 vs 2) ===
diff_FS_eV=4.528259888092377e-05
diff_RQM_eV=0.00013584779664277174
Ratio RQM/FS: 3.0000000000000093

=== TESTS: weight_mode=time, kappa_coeff=1.4142135623730951 ===
n= 2  pair(k1,k2)=(1,2)  diff_RQM_eV=1.3584779664277174e-04  diff_FS_eV=4.5282598880923773e-05  ratio(RQM/FS)=3.0000000000000093e+00
n= 3  pair(k1,k2)=(1,3)  diff_RQM_eV=5.3668265340353017e-05  diff_FS_eV=1.3417066335088522e-05  ratio(RQM/FS)=3.9999999999999201e+00
n= 4  pair(k1,k2)=(1,4)  diff_RQM_eV=2.5471461870519296e-05  diff_FS_eV=5.6603248601154707e-06  ratio(RQM/FS)=4.4999999999999432e+00
n= 5  pair(k1,k2)=(1,5)  diff_RQM_eV=1.3910814376217885e-05  diff_FS_eV=2.8980863283791216e-06  ratio(RQM/FS)=4.7999999999993452e+00

=== TESTS: weight_mode=time, kappa_coeff=1.4142135623730951 ===
n= 2  pair(k1,k2)=(1,2)  diff_RQM_eV=1.3584779664277174e-04  diff_FS_eV=4.5282598880923773e-05  ratio(RQM/FS)=3.0000000000000093e+00
n= 3  pair(k1,k2)=(2,3)  diff_RQM_eV=1.3417066335814104e-05 